# Assignment 3: RECOMMENDER SYSTEM FOR RECOMMENDING MOVIES

In [1]:
import pandas as pd
import numpy as np

from surprise.model_selection import train_test_split
from surprise import KNNWithMeans, Dataset, accuracy, Reader

In [2]:
link = pd.read_csv(r'D:\NAYANA\Term 2\ML\Assignment\Assignment 3\ml-latest-small\links.csv')
print(link.shape)
link.head()

(9742, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
movie = pd.read_csv(r'D:\NAYANA\Term 2\ML\Assignment\Assignment 3\ml-latest-small\movies.csv')
print(movie.shape)
movie.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating = pd.read_csv(r'D:\NAYANA\Term 2\ML\Assignment\Assignment 3\ml-latest-small\ratings.csv')
print(rating.shape)
rating.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
tag = pd.read_csv(r'D:\NAYANA\Term 2\ML\Assignment\Assignment 3\ml-latest-small\tags.csv')
print(tag.shape)
tag.head()

(3683, 4)


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


# 1. Create recommender system using ratings.csv file. 

In [6]:
reader = Reader(rating_scale = (1, 5))
data = Dataset.load_from_df(rating[['userId', 'movieId', 'rating']], reader)

In [7]:
[trainset, testset] = train_test_split(data, test_size = 0.15, shuffle = True)

recom = KNNWithMeans(k = 70, sim_options = {'Name' : 'Cosine', 'user_based' : True})

recom.fit(data.build_full_trainset())

test_pred = recom.test(testset)

RMSE = accuracy.rmse(test_pred)

print('RMSE = ',RMSE)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7293
RMSE =  0.7293168671753696


# 2. Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input).

In [8]:
user_id = int(input('Enter user ID : '))
movie_id = int(input('Enter movie ID : '))
print('The rating for User ID : {0} and Movie ID {1} is {2}'.format(user_id, movie_id, round((recom.predict(user_id,movie_id)[3]),2)))


Enter user ID : 600
Enter movie ID : 200
The rating for User ID : 600 and Movie ID 200 is 3.5


# 3. Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5.

In [16]:
recom.predict(User_id, Movie_id)[3]


3.501556983616962

In [15]:
# Creating an empty dictionary to append userid and movie id as key value pairs

movies = {}
User_id = int(input('Enter the User ID : '))

for Movie_id in range(1,193609):
    if recom.predict(User_id, Movie_id)[3] > 2.5:
        movies[Movie_id] = recom.predict(User_id, Movie_id)[3]

# To find top 10 values in the dictionary  
  
from collections import Counter

k = Counter(movies)

common = k.most_common(10)

# Printing those 10 recomended movie

print('10 suggested movies for user {0} are listed below'.format(User_id))

# Suggesting 10  movies

for i in range(10):    
    print(i+1, '. ', list(movie[movie.movieId == common[i][0]]['title'])[0])
    

Enter the User ID : 600
10 suggested movies for user 600 are listed below
1 .  Bossa Nova (2000)
2 .  The Big Bus (1976)
3 .  Galaxy of Terror (Quest) (1981)
4 .  Looker (1981)
5 .  Alien Contamination (1980)
6 .  Master of the Flying Guillotine (Du bi quan wang da po xue di zi) (1975)
7 .  Unfaithfully Yours (1948)
8 .  Match Factory Girl, The (Tulitikkutehtaan tyttö) (1990)
9 .  Seve (2014)
10 .  Villain (1971)


# 4. Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.

In [17]:
# To find the mean rating for all the movies

avg_rating = {}

for i in range(1, 193609):
    if (i == 33) | (i == 35) | (i==37):
        pass
    else:
        avg_rating[i] = round((rating[rating.movieId == i]['rating'].mean()), 2)
        

In [18]:
# Left Join on Rating dataframe

df_1 = pd.merge(rating[['movieId', 'rating']], tag[['movieId', 'tag']], how = 'left', on = 'movieId')

df_2 = pd.merge(df_1, link[['movieId', 'imdbId']], how = 'left', on = 'movieId')
df_3 = pd.merge(df_2, movie[['movieId', 'genres']], how = 'left', on = 'movieId')
df_3.head()


,movieId,rating,tag,imdbId,genres
0,1,4.0,pixar,114709,Adventure|Animation|Children|Comedy|Fantasy
1,1,4.0,pixar,114709,Adventure|Animation|Children|Comedy|Fantasy
2,1,4.0,fun,114709,Adventure|Animation|Children|Comedy|Fantasy
3,3,4.0,moldy,113228,Comedy|Romance
4,3,4.0,old,113228,Comedy|Romance


In [19]:
# Printing MovieID, IMDB ID, Average User Rating, Genres and Tag for all the movies of given User ID

ls = []

for i in range(10):
    ls.append(common[i][0])
    
df_4 = df_3[df_3['movieId'].isin(ls)]

# Dropping rating column and replacing that with avg_rating

df_4 = df_4.drop('rating',axis = 1)


In [20]:
pair = [(x,y) for x,y in avg_rating.items() if x in ls]

new_rating = []

for i in range(len(pair)):
    new_rating.append(pair[i][1])
    

In [21]:
# Sorting movieId in ascending order
# Adding a new column as Average Rating

df_5 = df_4.sort_values(by = 'movieId')
df_5['Avgerage Rating'] = new_rating
df_5


,movieId,tag,imdbId,genres,Avgerage Rating
153010,3567,NaN,180837,Comedy|Drama|Romance,5.0
207339,5490,NaN,74205,Action|Comedy,5.0
904,5746,NaN,82431,Action|Horror|Mystery|Sci-Fi,5.0
905,5764,NaN,82677,Drama|Horror|Sci-Fi|Thriller,4.5
908,6835,NaN,82000,Action|Horror|Sci-Fi,5.0
909,7899,NaN,72913,Action,4.5
274789,25947,NaN,40919,Comedy,5.0
255907,40491,depression,98532,Comedy|Drama,5.0
208682,132333,NaN,3149640,Documentary|Drama,5.0
150252,136850,NaN,67952,Crime|Drama|Thriller,5.0


# --------------------------------------------------- END ----------------------------------------------------